In [ ]:
# 01_data-cleaning.ipynb
# Initial Cleaning, Geoprocessing and Joining of Resilience Projects Data

# Libraries
import pickle   
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from pathlib import Path
from shapely.geometry import Point
import sys

sys.path.append(str(Path().resolve().parent / "src"))
from cleaning_functions import preview_df

# Paths
DATA_DIR = Path("../data/raw/")
TRACTS_PATH = DATA_DIR / "tl_2020_36_tract/tl_2020_36_tract.shp"

RESILIENCE_POINTS_PATH = (
    DATA_DIR / "resilience_projects/RecoveryResiliencyProjectMap_points_20250217.csv"
)

In [ ]:
## --- LOAD CENSUS TRACTS ---
# Load NYC Census Tracts
gdf_tracts = gpd.read_file(TRACTS_PATH)
gdf_tracts = gdf_tracts.to_crs(epsg=2263)

preview_df(gdf_tracts)

In [ ]:
## --- LOAD RESILIENCE PROJECTS DATA AS POINTS GEOMETRY ---
# Load Resilience Points Data
points = pd.read_csv(RESILIENCE_POINTS_PATH)

# Convert geometry
points["geometry"] = points["the_geom"].apply(wkt.loads)
gdf_points = gpd.GeoDataFrame(points, geometry="geometry", crs="EPSG:4326")

# Convert to NYC projected CRS
gdf_points = gdf_points.to_crs(epsg=2263)

# Clean column names
gdf_points.columns = gdf_points.columns.str.lower().str.strip()

preview_df(gdf_points)

In [ ]:
## --- JOIN RESILIENCE PROJECT POINTs TO CENSUS TRACTS ---
# Assign each resilience project to a census tract 
resilience_with_tract = gpd.sjoin(
    gdf_points, gdf_tracts[["GEOID", "geometry"]], how="left", predicate="within"
)

# Only completed projects
resilience_with_tract = resilience_with_tract.dropna(subset=["compdate"])

# Clean and extract project year
resilience_with_tract["year"] = (
    resilience_with_tract["compdate"]
    .str.extract(r"(\d{4})") 
    .astype(float)
    .astype("Int32") 
)

preview_df(resilience_with_tract)

In [ ]:
## --- SUMMARIZE RESILIENCE PROJECTS OVER TIME BY CENSUS TRACT ---
# Create tract-year level summary statistics
resilience_summary = (
    resilience_with_tract.groupby(["GEOID", "year"])
    .agg(
        has_resilience_project=("compdate", lambda x: 1), 
        proj_count=("compdate", "count"),
        projval_total=("projval", "sum"),
        #
        **{
            f"onenyc_{cat}": ("onenyc", lambda x, cat=cat: (x == cat).sum())
            for cat in resilience_with_tract["onenyc"].unique()
        },
    )
    .reset_index()
)

preview_df(resilience_summary)
print(resilience_summary["has_resilience_project"].value_counts())

In [ ]:
### --- LOAD MERGED AND GEOCODED ACS DATA ---
# LOAD ACS
with open(DATA_DIR / "../processed/acs_merged.pkl", "rb") as f:
    acs_merged = pickle.load(f)


In [ ]:
# --- PREP AND CLEAN ACS DATA ---
# Create derived share variables
acs_merged["poverty_rate"] = acs_merged["poverty_below"] / acs_merged["poverty_total"]
acs_merged["race_white_share"] = acs_merged["race_white"] / acs_merged["race_total"]
acs_merged["race_black_share"] = acs_merged["race_black"] / acs_merged["race_total"]
acs_merged["race_asian_share"] = acs_merged["race_asian"] / acs_merged["race_total"]
acs_merged["race_mixed_share"] = acs_merged["race_mixed"] / acs_merged['race_total']
acs_merged["race_native_pi_other_share"] = (
    acs_merged["race_native"]
    + acs_merged["race_pi"]
    + acs_merged["race_other"]
) / acs_merged["race_total"]

acs_merged["edu_no_diploma_share"] = (
    acs_merged["education_no_diploma"] / acs_merged["population"]
)
acs_merged["edu_high_school_share"] = (
    acs_merged["education_high_school"] / acs_merged["population"]
)
acs_merged["edu_associates_share"] = (
    acs_merged["education_associates"] / acs_merged["population"]
)
acs_merged["edu_bachelors_share"] = (
    acs_merged["education_bachelors"] / acs_merged["population"]
)
acs_merged["edu_masters_share"] = (
    acs_merged["education_masters"] / acs_merged["population"]
)
acs_merged["edu_professional_share"] = (
    acs_merged["education_professional"] / acs_merged["population"]
)
acs_merged["edu_doctorate_share"] = (
    acs_merged["education_doctorate"] / acs_merged["population"]
)
acs_merged["edu_college_plus"] = (
    acs_merged["education_bachelors"]
    + acs_merged["education_masters"]
    + acs_merged["education_professional"]
    + acs_merged["education_doctorate"]
)
acs_merged["edu_college_plus_share"] = (
    acs_merged["edu_college_plus"] / acs_merged["population"]
)

# Build tract-year base from ACS 
tract_years = acs_merged[["GEOID", "year"]].drop_duplicates().dropna()
tract_years["year"] = tract_years["year"].astype(int)

# Merge in full ACS demographic panel
tract_base = tract_years.merge(acs_merged, on=["GEOID", "year"], how="left")

# Merge in resilience summary
tract_base = tract_base.merge(resilience_summary, on=["GEOID", "year"], how="left")

# Fill NA values for resilience columns
tract_base["has_resilience_project"] = tract_base["has_resilience_project"].fillna(0).astype(int)
tract_base["proj_count"] = tract_base["proj_count"].fillna(0).astype(int)
tract_base["projval_total"] = tract_base["projval_total"].fillna(0)

# Fill NA in dummy columns like onenyc_*
onenyc_cols = [col for col in tract_base.columns if col.startswith("onenyc_")]
tract_base[onenyc_cols] = tract_base[onenyc_cols].fillna(0).astype(int)

# Get First Project Year Per Tract from Point-Level Data 
first_project_year = (
    resilience_with_tract.groupby("GEOID")["year"]
    .min()
    .reset_index()
    .rename(columns={"year": "first_project_year"})
)

# Merge into the Tract-Year Panel
gdf_tracts_with_resilience = tract_base.merge(
    first_project_year, on="GEOID", how="left"
)

# Create Post-Project Indicator
gdf_tracts_with_resilience["post_project"] = (
    (gdf_tracts_with_resilience["year"] >= gdf_tracts_with_resilience["first_project_year"]).fillna(False).astype(int)
)
# Years Since Project (set to 0 if not yet implemented) 
gdf_tracts_with_resilience["years_since_project"] = (
    gdf_tracts_with_resilience["year"]
    - gdf_tracts_with_resilience["first_project_year"]
)

gdf_tracts_with_resilience["years_since_project"] = gdf_tracts_with_resilience[
    "years_since_project"
].where(gdf_tracts_with_resilience["first_project_year"].notna())

gdf_tracts_with_resilience.loc[gdf_tracts_with_resilience["years_since_project"] < 0, "years_since_project"] = np.nan

# Cumulative Exposure Duration (integer)
gdf_tracts_with_resilience["project_exposure_duration"] = (
    gdf_tracts_with_resilience["years_since_project"].fillna(0).astype(int)
)

# Lagged Exposure Dummies (1 through 5 years post-project)
for lag in range(1, 6):
    gdf_tracts_with_resilience[f"post_project_lag{lag}"] = (
        (gdf_tracts_with_resilience["years_since_project"] == lag)
        .fillna(False)
        .astype(int)
    )

# Ensure the GeoDataFrame uses the 'geometry' column for geometry and set CRS to EPSG:2263
gdf_tracts_with_resilience = gpd.GeoDataFrame(
    gdf_tracts_with_resilience,
    geometry="geometry",
    crs="EPSG:2263"
)

preview_df(gdf_tracts_with_resilience)

In [ ]:
## --- LOAD EVICTIONS DATA ---
# Load Evictions Data
evictions = pd.read_csv(DATA_DIR / "evictions/Evictions_20250219.csv", dtype=str)

# Clean column names
evictions.columns = evictions.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('/', '_')

# Clean column types
evictions['executed_date'] = pd.to_datetime(evictions['executed_date'], format='%m/%d/%Y', errors='coerce')
evictions['latitude'] = pd.to_numeric(evictions['latitude'], errors='coerce')
evictions['longitude'] = pd.to_numeric(evictions['longitude'], errors='coerce')
evictions['year'] = evictions['executed_date'].dt.year

# Drop rows with missing coordinates or dates
evictions_clean = evictions.dropna(subset=['latitude', 'longitude', 'executed_date'])


# Create geometry column from lat/lon (if not already a GeoDataFrame)
gdf_evictions = gpd.GeoDataFrame(
    evictions_clean.copy(),
    geometry=gpd.points_from_xy(
        evictions_clean["longitude"], evictions_clean["latitude"]
    ),
    crs="EPSG:4326",
)

# Project to match the tracts CRS
gdf_evictions = gdf_evictions.to_crs(gdf_tracts.crs)

# Spatial join to assign GEOID from tracts
gdf_evictions = gpd.sjoin(
    gdf_evictions, gdf_tracts[["GEOID", "geometry"]], how="left", predicate="within"
)

# Group by GEOID and year to count evictions per tract-year
evictions_by_tract_year = (
    gdf_evictions.groupby(["GEOID", "year"]).size().reset_index(name="evictions_count")
)

# Merge evictions into resilience panel by GEOID and year
gdf_tracts_with_resilience = gdf_tracts_with_resilience.merge(
    evictions_by_tract_year, on=["GEOID", "year"], how="left"
)

# Fill tracts with no recorded evictions with 0
gdf_tracts_with_resilience["evictions_count"] = (
    gdf_tracts_with_resilience["evictions_count"].fillna(0).astype(int)
)

# Separate by Residential vs Commercial Evictions 
eviction_type_summary = (
    gdf_evictions.groupby(["GEOID", "year", "residential_commercial"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(
        columns={
            "Residential": "eviction_residential",
            "Commercial": "eviction_commercial",
        }
    )
)

# Merge type-specific counts
gdf_tracts_with_resilience = gdf_tracts_with_resilience.merge(
    eviction_type_summary, on=["GEOID", "year"], how="left"
)

# Fill missing type counts with 0
for col in ["eviction_residential", "eviction_commercial"]:
    if col in gdf_tracts_with_resilience.columns:
        gdf_tracts_with_resilience[col] = (
            gdf_tracts_with_resilience[col].fillna(0).astype(int)
        )
    else:
        gdf_tracts_with_resilience[col] = (
            0  # if column missing (e.g., only residential data)
        )

# Calculate eviction rate (residential per 1,000 residents)
gdf_tracts_with_resilience["eviction_rate_per_1000"] = (
    gdf_tracts_with_resilience["eviction_residential"]
    * 1000
    / gdf_tracts_with_resilience["population"]
)

# Handle infinite or undefined rates
gdf_tracts_with_resilience["eviction_rate_per_1000"] = gdf_tracts_with_resilience[
    "eviction_rate_per_1000"
].replace([np.inf, -np.inf], np.nan)

# Flag High Eviction Years: above 90th percentile by tract 
def flag_high_evictions(group):
    threshold = group["eviction_residential"].quantile(0.90)
    group["high_eviction_year"] = (group["eviction_residential"] >= threshold).astype(
        int
    )
    return group

gdf_merged = gdf_tracts_with_resilience.groupby(
    "GEOID", group_keys=False
).apply(flag_high_evictions)

gdf_merged = gpd.GeoDataFrame(gdf_merged, geometry="geometry", crs="EPSG:2263")

# Handle Invalid Geometries (if any)
gdf_merged = gdf_merged[gdf_merged.is_valid]  

gdf_merged["GEOID"] = gdf_merged["GEOID"].astype(int)
gdf_merged["year"] = gdf_merged["year"].astype(int)

# Fill first_project_year with 0 to indicate "never treated"
gdf_merged["first_project_year"] = (
    gdf_merged["first_project_year"].fillna(0).astype(int)
)

# Fill years_since_project with -1 to indicate "not treated yet"
gdf_merged["years_since_project"] = (
    gdf_merged["years_since_project"].fillna(-1).astype(int)
)

# Extract latitude and longitude from geometry
gdf_merged["latitude"] = gdf_merged.geometry.centroid.y
gdf_merged["longitude"] = gdf_merged.geometry.centroid.x

preview_df(gdf_merged)

In [ ]:
# --- EXPORT DATA ---

# As Pickle
gdf_merged.to_pickle(DATA_DIR / "../modeling/gdf_tracts.pkl")

# As CSV
gdf_for_csv = gdf_merged.drop(columns=["geometry"])
gdf_merged.to_csv(
    DATA_DIR / "../modeling/tracts.csv", index=False
)